<center><h2><strong><font color="blue">WFH 2024 idBigData - Word Embedding</font></strong></h2></center>

<img alt="" src="images/covers/cover_taudata_uin.jpg"/>

<center><h2><strong><font color="blue">Referensi</font></strong></h2></center>

* https://taudata.blogspot.com/2022/04/nlptm-05.html
* https://www.youtube.com/watch?v=viZrOnJclY0
* https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0
* https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

<center><h1><strong><font color="blue">One-Hot VS Word Embedding</font></strong></h1></center>

<img alt="" src="images/BoW_VS_WordEmbedding.png" style="height:250px;" />

<img alt="" src="images/onehot-vs-embedding.png" style="height:250px;" />

<center><h1><strong><font color="blue">Word Embedding Models</font></strong></h1></center>

<img alt="" src="images/DL/Word-Embedding-models.png" style="height:400px;" />

* https://medium.com/nlplanet/two-minutes-nlp-11-word-embeddings-models-you-should-know-a0581763b9a9

<img alt="" src="images/3_word2vec_example.png" style="height:400px; width:667px" />

<center><h1><strong><font color="blue">Word2Vec</font></strong></h1></center>

<img alt="" src="images/DL/Word2Vec.png" style="height:200px;" />

* Dikembangkan oleh Tomas Mikolov - Google :</p>
* Goldberg, Yoav; Levy, Omer. &quot;word2vec Explained: Deriving Mikolov et al.&#39;s Negative-Sampling Word-Embedding Method&quot;.&nbsp;<a href="https://en.wikipedia.org/wiki/ArXiv">arXiv</a>:<a href="https://arxiv.org/abs/1402.3722">1402.3722</a>

<center><h1><strong><font color="blue">FastText VS Word2Vec</font></strong></h1></center>

<ul>
	<li>Menggunakan Sub-words: app, ppl, ple - apple</li>
	<li>Paper:&nbsp;https://arxiv.org/abs/1607.04606&nbsp;&nbsp;</li>
	<li>Website:&nbsp;https://fasttext.cc/</li>
	<li>Source:&nbsp;https://github.com/facebookresearch/fastText&nbsp;</li>
</ul>


<img alt="" src="images/fastText-vs-Word2Vec.png" style="height: 400px;" />

<img alt="" src="images/DL/word2Vec-to-visualization-via-dimensional-reduction.png" style="height:400px;" />

<img alt="" src="images/3_cosine.png" style="height:300px;" />

### Hati-hati Cosine adalah similarity bukan distance
Hal ini akan mempengaruhi interpretasi

In [2]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm 

try:
    import google.colab; IN_COLAB = True
    print("Installing the required modules")
    !pip install torch, nltk, requests, dewiki, unidecode --q
    print("preparing directories and assets")
    !mkdir data images output models
    #!wget https://raw.githubusercontent.com/taudata...
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

import re, gc, os, nltk, requests
import torch, torch.nn as nn, torch.optim as optim
import bz2,re,dewiki, html.parser
from unidecode import unidecode
from html import unescape
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
from bs4 import Comment
from collections import Counter
from nltk.tokenize import word_tokenize

nltk.download('punkt')
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

nSample = 100000 # Sample dokumen agar memory dan komputasi Tidak terlalu besar

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
print("Initializing CPU and-or GPU ... ")
gc.collect()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
torch.cuda.empty_cache()
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the {}:'.format(device.type.upper()), torch.cuda.get_device_name(0))
    print("WARNING!!!... Walau menggunakan GPU Model ini tetap membutuhkan memori dan komputasi yang besar.")
else:
    device = torch.device("cpu")
    print("WARNING!!!... No GPU available, using the '{}' instead. NOT Recommended, it is going to take forever for the code to finish!".format(device.type.upper()))
    print("Setidaknya gunakan sample Max 30.000 dokumen saja untuk pembelajaran.")

In [ ]:
# Fungsi untuk membersihkan teks
def cleanTags(text):
    text = re.sub(r'<[^>]+>', ' ', text) # Remove HTML tags
    text = re.sub(r'<\?php.*?\?>', '', text, flags=re.DOTALL)# Remove PHP tags
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)# Remove JavaScript
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL) # Remove CSS
    return text

def cleanText(text, minWord=2, maxWord=20):
    t = dewiki.from_string(str(text))
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    t = re.sub(pattern,' ',t) #remove http urls if any
    pattern = re.compile(r'ftp[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    t = re.sub(pattern,' ',t) #remove ftp urls if any
    t = re.sub("[\w]+@[\w]+\.[c][o][m]", "", t) #remove emails
    t = cleanTags(t)
    t = unescape(unidecode(t.lower().strip()))
    t = t.replace('\n', ' ').replace('\r', ' ')
    #listKata = [tok for tok in TextBlob(t).words if (str(tok) not in stopWordsID)]
    pattern = r"[^a-zA-Z0-9 .,_-]"
    t = re.sub(pattern, " ", t) # remove all Symbols except : {, . _ -}
    t = re.sub(r'\s+', ' ', t) # remove multiple spaces
    t = " ".join([w for w in t.split() if len(w)>=minWord and len(w)<=maxWord])
    return t.strip()

def getContents(dStr):
    judul, pageID, body = None, None, None
    if len(dStr)>7:
        awalTag=re.search('<text.*?>',dStr).group(0)
        awal=dStr.find(awalTag);akhir=dStr.find('</text>')
        if awal>=0 and akhir>=0:
            body=dStr[awal+len(awalTag):akhir]
        dStr=dStr[:awal] # id and title came before <text>, no need to search the whole text again
    if len(dStr)>7:
        awal=dStr.find('<title>');akhir=dStr.find('</title>')
        if awal>=0 and akhir>=0:
            judul=dStr[awal+len('<title>'):akhir]
        awal=dStr.find('<id>');akhir=dStr.find('</id>')
        if awal>=0 and akhir>=0:
            pageID=dStr[awal+len('<id>'):akhir]
    return judul,pageID,body

def getCategories(body):#if categories === if categories is not empty
    categories=[];dTag1='[[Category:';dTag2=']]';n1=len(dTag1);n2=len(dTag2);
    if type(body) is str:
        while True and len(body)>12:
            awal=body.find(dTag1);akhir=body.find(dTag2)
            if awal>=0 and akhir>=0:
                cat=body[awal+n1:akhir]
                if len(cat)>1:
                    categories.append(cat)
                body=body[akhir+n2:]
            else:
                break
    return ','.join(categories)

# Buat pasangan konteks dan target
def generate_context_target_pairs(tokens, context_size):
    pairs = []
    for token_list in tqdm(tokens):
        token_list = [word for word in token_list if word in word_to_idx]
        for i, word in enumerate(token_list):
            target = word_to_idx[word]
            context = [word_to_idx[token_list[i + j]] for j in range(-context_size, context_size + 1) if 0 <= i + j < len(token_list) and j != 0]
            pairs.extend((context_word, target) for context_word in context)
    return pairs

# Dataset dan DataLoader
class Word2VecDataset(torch.utils.data.Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx]
    
def get_word_embedding(word, GPU=True):
    idx = word_to_idx.get(word)
    if idx is not None:
        with torch.no_grad():
            if GPU:
                embedding = model.embeddings(torch.tensor([idx]).cuda())
            else:
                embedding = model.embeddings(torch.tensor([idx]))
        return torch.flatten(target_embedding).cpu()
    else:
        print(f"Word '{word}' not in vocabulary")
        return None
    
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def find_most_similar(word, top_n=5, GPU=True):
    target_embedding = get_word_embedding(word, GPU=GPU)
    if target_embedding is None:
        return

    similarities = []
    for other_word in tqdm(vocab):
        if other_word != word:
            other_embedding = get_word_embedding(other_word, GPU=GPU)
            similarity = cosine_similarity(target_embedding, other_embedding)
            similarities.append((other_word, similarity))
    
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

"Done"

# <center><font color="blue">Load Dataset Wikipedia Indonesia Latest Dump</font></center>

* Keterangan tentang wiki dumps: https://en.wikipedia.org/wiki/Wikipedia:Database_download
* Wikipedia Indonesia dumps: https://id.wikipedia.org/wiki/Wikipedia:Unduh_basis_data

<img alt="" src="images/Wiki_logo.jpeg" style="height: 400px;" />

## <center><font color="red">Hati-hati akan mendownload data Text yang cukup besar (Hampir 1Gb pada Juli 2024)</font></center>

In [ ]:
# Unduh data Wikipedia bahasa Indonesia
url = "https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2"
dataFile = "data/idwiki-latest-pages-articles.xml.bz2"

if not os.path.isfile(dataFile) and not os.path.isfile(dataFile.replace(".xml.bz2", ".zip")):
    print("Start Downloading Wikipedia Dumps Data : '{}' ...".format(dataFile), flush=True, end=" ")
    response = requests.get(url, stream=True)
    with open(dataFile, "wb") as file:
        for chunk in tqdm(response.iter_content(chunk_size=1024)):
            file.write(chunk)
            
if os.path.isfile(dataFile.replace(".xml.bz2", ".zip")):
    print("Reading the previously parsed Wikipedia XML dump: {}".format(dataFile.replace(".xml.bz2", ".zip")))
    df = pd.read_csv(dataFile.replace(".xml.bz2", ".zip"), compression='zip')
else:
    print("Reading the Wikipedia XML dump: {}".format(dataFile))
    print("Please wait, it might take awhile .... ")
    id_,  dPage = 0, []
    df = {"id_":[], "category":[], "title":[], "content":[]}
    with bz2.BZ2File(dataFile, "r") as f:
        for line in tqdm(f):
            line=line.decode('UTF-8')
            if '<page>' in line:
                dPage=[]
            elif '</page>' in line:
                try:
                    judul, _, body = getContents(' '.join(dPage))
                    cat = cleanText(getCategories(body))
                    judul = cleanText(judul)
                    body = cleanText(body)
                    if len(str(judul))>5 and len(str(body))>5:
                        df['id_'].append(id_); id_ += 1
                        df['category'].append(cat)
                        df['title'].append(judul)
                        df['content'].append(body)
                except Exception as err_:
                    #print(err_)
                    pass
            else:
                dPage.append(line)
    df = pd.DataFrame(df)
    print("Finished, Compressing the dataFrame for future operations ... ")
    df.to_csv(dataFile.replace(".xml.bz2", ".zip"), index=False, compression={'method':'zip', 'compresslevel':9})

print(df.shape)
df.head()

In [ ]:
print("Sampling {} documents.".format(nSample))
df = df.sample(nSample)

print("Joining Text ... ")
texts = [" ".join([t, c]) for t,c in tqdm(zip(df.title, df.content))]
del df # Free some memory
texts[0]

In [ ]:
# Tokenisasi teks
print("Tokenizing ... ")
tokens = [word_tokenize(text) for text in tqdm(texts)]
del texts # Free Some memory
"\nToken Length = {}\n".format(len(tokens)), tokens[0][:7]

# <center><font color="blue">Menyiapkan Struktur Data</font></center>

In [ ]:
# Buat korpus kata
all_tokens = [token for sublist in tqdm(tokens) for token in sublist]
vocab = Counter(all_tokens)
vocab = [word for word, freq in vocab.items() if freq>=5 and len(word)>2]  # Filter kata yang jarang muncul

# Peta kata ke indeks dan sebaliknya
word_to_idx = {word: i for i, word in tqdm(enumerate(vocab))}
idx_to_word = {i: word for i, word in tqdm(enumerate(vocab))}

context_size = 2
pairs = generate_context_target_pairs(tokens, context_size)

dataset = Word2VecDataset(pairs)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

vocab_size = len(vocab)
print(vocab[:5])
del all_tokens, tokens # free some memory

# <center><font color="blue">Model Word2Vec</font></center>

In [ ]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, vocab_size)

    def forward(self, context_word):
        embeds = self.embeddings(context_word)
        out = self.linear1(embeds)
        return out

embedding_dim = 50
if device.type.lower().strip()=="cpu":
    model = Word2Vec(vocab_size, embedding_dim)
    criterion = nn.CrossEntropyLoss()
else:
    model = Word2Vec(vocab_size, embedding_dim).cuda()
    criterion = nn.CrossEntropyLoss().cuda()
    
optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer

# <center><font color="blue">Training</font></center>

### Tambah epoch di kasus nyata dan gunakan early stopping

In [ ]:
nEpoch = 5 # Change to higher value in realworld scenario
fmodel = 'data/models/word2Vec-wikiID.pth'
try:
    model = torch.load(fmodel)
    print("Model Loaded from previous Run.")
except:
    print("Training the model, please wait, it will take a while .. ")
    for epoch in tqdm(range(nEpoch)):
        total_loss = 0
        for context_word, target in dataloader:
            if device.type.lower().strip()=="cpu":
                context_word, target = torch.tensor(context_word), torch.tensor(target)
            else:
                context_word, target = torch.tensor(context_word).cuda(), torch.tensor(target).cuda()
                
            optimizer.zero_grad()
            outputs = model(context_word)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")
    print("Finished!, Saving The Model... ")
    torch.save(model, fmodel)

# <center><font color="blue">Contoh Penggunaan</font></center>

In [ ]:
if device.type.lower().strip()=="cpu":
    GPU = False
else:
    GPU = True
word = 'agama'
most_similar_words = find_most_similar(word, top_n=7, GPU=True)
print(f"Kata-kata yang paling mirip dengan '{word}':")
for similar_word, similarity in most_similar_words:
    print(f"{similar_word}: {similarity}")

# <center><font color="blue"> Akhir Modul </font></center>

<hr />
<img alt="" src="images/meme-cartoon/oprah-meme-everything-gets-a-vector-word-embedding.png" style="height: 400px;"/>